In [1]:
import os
from dotenv import load_dotenv
# load_dotenv() os.getenv("GOOGLE_API_KEY")
import google.generativeai as genai
import json
import pandas as pd
import traceback

e:\SOFTWARES\Anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-pro')


In [3]:
response=model.generate_content("Who is the fouder of google what was the founders first discovery how was google formed")

In [4]:
print(response.text)

**Founders of Google:**

**1. Larry Page:**

   * Born on March 26, 1973, in East Lansing, Michigan.
   * Earned a Bachelor of Science degree in Computer Engineering from the University of Michigan.
   * Completed a Master of Science degree in Computer Science from Stanford University.

**2. Sergey Brin:**

   * Born on August 21, 1973, in Moscow, Russia.
   * Immigrated to the United States with his family at the age of 6.
   * Earned a Bachelor of Science degree in Mathematics and Computer Science from the University of Maryland, College Park.
   * Completed a Master of Science degree in Computer Science from Stanford University.

**First Discovery:**

The founders' first discovery as graduate students at Stanford University was the creation of a search engine called "BackRub." This search engine used a novel algorithm called PageRank, which ranked web pages based on their importance and relevance to a search query.

**Formation of Google:**

* In 1998, Page and Brin formally incorpo

In [5]:
# from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
# from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
file_paths=r"G:\LILTHOMA\Learn\LLM\MCQ_Generator_using_gemini_pro\NIPS-2017-attention-is-all-you-need-Paper.pdf"
pdf_reader=PyPDF2.PdfReader(file_paths)

In [9]:
pdf_reader

In [7]:
from langchain_google_genai import GoogleGenerativeAI
api_key=os.getenv("GOOGLE_API_KEY")

In [8]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros of Python**

* **Easy to learn:** Python is a very easy language to learn, even for beginners. The syntax is simple and straightforward, and there are plenty of resources available to help you get started.
* **Versatile:** Python can be used for a wide variety of tasks, from simple scripts to complex applications. It is also a good choice for beginners because it can be used for both web development and desktop applications.
* **Highly portable:** Python is a very portable language, which means that it can be run on a variety of platforms, including Windows, Mac, and Linux. This makes it a good choice for developing applications that need to be deployed on multiple platforms.
* **Open source:** Python is an open source language, which means that it is free to use and modify. This makes it a good choice for developers who want to have complete control over their code.
* **Large community:** Python has a large and active community of users and developers, which means that there is

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [12]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [13]:

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
#Sequential chain using output from quiz_chain as input for review _cahin
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path=r"G:\LILTHOMA\Learn\LLM\MCQ_Generator_using_gemini_pro\texts.txt"
file_path

'G:\\LILTHOMA\\Learn\\LLM\\MCQ_Generator_using_gemini_pro\\texts.txt'

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=5 
SUBJECT="machine learning"
TONE="hard"

In [21]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Supervised Machine Learning
Supervised learning is the types of machine learning in which machines are trained using well "labelled" training data, and on basis of that data, machines predict the output. The labelled data means some input data is already tagged with the correct output.

In supervised learning, the training data provided to the machines work as the supervisor that teaches the machines to predict the output correctly. It applies the same concept as a student learns in the supervision of the teacher.

Supervised learning is a process of providing input data as well as correct output data to the machine learning model. The aim of a supervised learning algorithm is to find a mapping function to map the input variable(x) with the output variable(y).

In the real-world, supervised learning can be used for Risk Assessment, Image classification, Fraud Detection, spam filt


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Text:Supervised Machine Learning
Supervised learning is the types of machine learning in which machines are trained using well "labelled" training data, and on basis of that data, machines predict the output. The labelled data means some input data is already tagged with the correct output.

In supervised learning, the training data provided to the machines work as the supervisor that teaches the machines to predict the output correctly. It applies the same concept as a student learns in the supervision of the teacher.

Supervised learning is a process of providing input data as well as correct output data to the machine learning model. The aim of a supervised learning algorithm is to find a mapping function to map the input variable(x) with the output variable(y).

In the real-world, supervised learning can be used for Risk Assessment, Image classification, Fraud Detection, spam filtering, etc.
You are an 

In [22]:
response

{'text': 'Supervised Machine Learning\nSupervised learning is the types of machine learning in which machines are trained using well "labelled" training data, and on basis of that data, machines predict the output. The labelled data means some input data is already tagged with the correct output.\n\nIn supervised learning, the training data provided to the machines work as the supervisor that teaches the machines to predict the output correctly. It applies the same concept as a student learns in the supervision of the teacher.\n\nSupervised learning is a process of providing input data as well as correct output data to the machine learning model. The aim of a supervised learning algorithm is to find a mapping function to map the input variable(x) with the output variable(y).\n\nIn the real-world, supervised learning can be used for Risk Assessment, Image classification, Fraud Detection, spam filtering, etc.',
 'number': 5,
 'subject': 'machine learning',
 'tone': 'hard',
 'response_jso

In [24]:
quiz=response.get("quiz")
quiz=quiz.replace('```json\n', '').replace('```', '')


In [25]:
quiz=json.loads(quiz)
quiz

{'1': {'mcq': 'In supervised learning, the training data is labeled. What is the other name of labeled data?',
  'options': {'a': 'unlabeled data',
   'b': 'labeled data',
   'c': 'training data',
   'd': 'testing data'},
  'correct': 'b'},
 '2': {'mcq': 'In supervised learning, the model is trained using labeled data. What is the goal of the model?',
  'options': {'a': 'to learn the mapping function',
   'b': 'to predict the output',
   'c': 'to find the correct answer',
   'd': 'to find the correct output'},
  'correct': 'a'},
 '3': {'mcq': 'Supervised learning is used for various applications. Which of the following is not an application of supervised learning?',
  'options': {'a': 'Risk assessment',
   'b': 'Image classification',
   'c': 'Fraud detection',
   'd': 'Natural Language Processing'},
  'correct': 'd'},
 '4': {'mcq': 'Supervised learning is a type of machine learning. What is the other type of machine learning?',
  'options': {'a': 'Unsupervised learning',
   'b': 'Rein

In [28]:
for key,value in quiz.items():
    mcq=value['mcq']
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    print(options)

a: unlabeled data | b: labeled data | c: training data | d: testing data
a: to learn the mapping function | b: to predict the output | c: to find the correct answer | d: to find the correct output
a: Risk assessment | b: Image classification | c: Fraud detection | d: Natural Language Processing
a: Unsupervised learning | b: Reinforcement learning | c: Deep learning | d: All of the above
a: to evaluate the model | b: to train the model | c: to test the model | d: to validate the model


In [29]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [30]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)